In [ ]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import catboost as cb
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.stats import uniform, randint
import warnings

In [ ]:
#Define random seed for result reproduction
random_seed = 30
np.random.seed(random_seed)

In [4]:
# Load the pickle file
pickle_path = "../data/processed/final_data.pck"

with open(pickle_path, 'rb') as f:
    data = pickle.load(f)
    
warnings.filterwarnings('ignore')

In [ ]:
# separate features and target variable
X = df.drop(target, axis=1)
y = df[target]

# remove columns with any missing values
X = X.dropna(axis=1, how='any')

# one hot encode categorical variables
X = pd.get_dummies(X, drop_first=True)

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

Linear regression

In [ ]:
# Modeling
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)

y_pred = linear_regressor.predict(X_test)

rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred))
r2_lr = r2_score(y_test, y_pred)

print("Linear regression results")
print(f"RMSE: {rmse_lr}")
print(f"R^2: {r2_lr}")

Random forest

In [ ]:
# Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=random_seed)
rf.fit(X_train, y_train)

# predictions for the test set
y_pred_rf = rf.predict(X_test)

# calculating r2 for random forest
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest results")
print(f"RMSE: {rmse_rf}")
print(f"R^2: {r2_rf}")

Optimalized random forest

In [ ]:
# RandomForestRegressor
orf_regressor = RandomForestRegressor(random_state=random_seed, n_jobs=-1)


param_grid_orf = {
    'n_estimators': [200, 300, 500],
    'max_features': [4,6],
    'min_samples_split': [3,10],
    'bootstrap': [True, False]
}

# GridSearchCV
grid_search_orf = GridSearchCV(estimator=orf_regressor, param_grid=param_grid_orf, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)

# Training model
grid_search_orf.fit(X_train, y_train)

# Best parametres
best_params_orf = grid_search_orf.best_params_
print(f"Best params: {best_params_orf}")

# Best model
best_model_orf = grid_search_orf.best_estimator_

# Prediction with best model
predictions_orf = best_model_orf.predict(X_test)

# Model evaluation
rmse_orf = np.sqrt(mean_squared_error(y_test, predictions_orf))
r2_orf = r2_score(y_test, predictions_orf)

print("Optimized Random Forest results")
print(f"RMSE: {rmse_orf}")
print(f"R^2: {r2_orf}")

In [ ]:
# Feature importance
importances = rf.feature_importances_
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Plotting
plt.figure(figsize=(10, 8))
plt.barh(feature_importances['Feature'], feature_importances['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.show()

feature_importances.head(10)

XGBoost

In [ ]:
xgb1_regressor = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators=100, random_state=random_seed)
xgb1_regressor.fit(X_train, y_train)
y_pred_xgb1 = xgb1_regressor.predict(X_test)

rmse_xgb1 = np.sqrt(mean_squared_error(y_test, y_pred_xgb1))
r2_xgb1 = r2_score(y_test, y_pred_xgb1)

print("XGBoost results")
print(f"RMSE: {rmse_xgb1}")
print("R2: {:.2f}".format(r2_xgb1))

Optimized XGBoost

In [ ]:
# Custom XGBoost wrapper to include early stopping
class XGBRegressorEarlyStopping(xgb.XGBRegressor):
    def __init__(self, early_stopping_rounds=10, **kwargs):
        super().__init__(**kwargs)
        self.early_stopping_rounds = early_stopping_rounds

    def fit(self, X, y, eval_set=None, **kwargs):
        if eval_set is None:
            eval_set = [(X, y)]
        return super().fit(X, y, eval_set=eval_set, **kwargs)

# Pipeline with scaling and model
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', XGBRegressorEarlyStopping(objective='reg:squarederror', random_state=random_seed, early_stopping_rounds=10))
])

# Parameter distribution for randomized search
param_dist = {
    'xgb__n_estimators': randint(50, 200),
    'xgb__learning_rate': uniform(0.01, 0.3),
    'xgb__max_depth': randint(3, 10),
    'xgb__min_child_weight': randint(1, 6),
    'xgb__subsample': uniform(0.5, 0.5),
    'xgb__colsample_bytree': uniform(0.5, 0.5),
    'xgb__gamma': uniform(0, 0.4),
    'xgb__reg_alpha': uniform(0, 1),
    'xgb__reg_lambda': uniform(0.5, 2)
}

# Randomized search
random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist, n_iter=50, 
                                   cv=3, n_jobs=-1, verbose=2, scoring='r2', random_state=random_seed)
random_search.fit(X_train, y_train)

# Best model
best_pipeline = random_search.best_estimator_

# Predictions and evaluation
y_pred_xgb = best_pipeline.predict(X_test)
r2_xgb = r2_score(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

print("Best R2: {:.2f}".format(r2_xgb))
print("Best RMSE: {:.2f}".format(rmse_xgb))
print("Best Parameters: ", random_search.best_params_)


Catboost

In [ ]:
# CatBoostRegressor
cb_regressor = cb.CatBoostRegressor(random_seed=random_seed, silent=True)

# RandomizedSearch parameters
param_distributions_cb = {
    'iterations': [500, 1000, 1500],
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'bootstrap_type': ['Bayesian', 'Bernoulli', 'MVS'],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bylevel': [0.8, 0.9, 1.0]
}

# RandomizedSearchCV
random_search_cb = RandomizedSearchCV(
    estimator=cb_regressor,
    param_distributions=param_distributions_cb,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=random_seed
)

random_search_cb.fit(X_train, y_train)

# Best params
best_params_cb = random_search_cb.best_params_
print(f"Best params: {best_params_cb}")

# Best model
best_model_cb = random_search_cb.best_estimator_

# Prediction
predictions_cb = best_model_cb.predict(X_test)

# Model evaluation
rmse_cb = np.sqrt(mean_squared_error(y_test, predictions_cb))
r2_cb = r2_score(y_test, predictions_cb)

print("CatBoost results")
print(f"RMSE: {rmse_cb}")
print(f"R^2: {r2_cb}")

In [ ]:
# Model comparison
results = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest', "Optimized Random Forest", 'CatBoost', 'XGBoost', "Optimized XGBoost"],
    'RMSE': [rmse_lr, rmse_rf, rmse_orf, rmse_cb, rmse_xgb1 , rmse_xgb],
    'R²': [r2_lr, r2_rf, r2_orf , r2_cb, r2_xgb1 , r2_xgb]
})

print(results)

In [ ]:
# Determine the best RMSE and R²
best_rmse_index = results['RMSE'].idxmin()
best_r2_index = results['R²'].idxmax()

# RMSE
plt.figure(figsize=(10, 6))
bars = plt.bar(results['Model'], results['RMSE'], color='skyblue')
bars[best_rmse_index].set_color('red')
plt.title('Model Comparison: RMSE')
plt.xlabel('Model')
plt.ylabel('RMSE')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# R²
plt.figure(figsize=(10, 6))
bars = plt.bar(results['Model'], results['R²'], color='skyblue')
bars[best_r2_index].set_color('red')
plt.title('Model Comparison: R²')
plt.xlabel('Model')
plt.ylabel('R²')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()